# Disclaimer & Copyright

Copyright 2024 Forusone : shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Gemini - System instructions
* System instructions are like a preamble that you add before the LLM gets exposed to any further instructions from the user. It lets users steer the behavior of the model based on their specific needs and use cases.
* For example, you can include things like the role or persona, contextual information, and formatting instructions.
* Refer to https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/system-instructions



# Configuration
## Install python packages
* Vertex AI SDK for Python
  * https://cloud.google.com/python/docs/reference/aiplatform/latest

In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 11.7 MB/s eta 0:00:00


In [2]:
from IPython.display import display, Markdown

## Authentication to access to the GCP & Google drive

* Use OAuth to access the GCP environment.
 * Refer to the authentication methods in GCP : https://cloud.google.com/docs/authentication?hl=ko

In [3]:
#  For only colab to authenticate to get an access to the GCP.
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

* Mount to the google drive to access the .ipynb files in the repository.



In [4]:
# To access contents in Google drive

if "google.colab" in sys.modules:
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


# Execute the example
## Set the environment on GCP Project
* Configure project information
  * Model name : LLM model name : https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models
  * Project Id : prodect id in GCP
  * Region : region name in GCP

In [5]:
MODEL_NAME="gemini-1.5-flash"
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"

### Vertex AI initialization
Configure Vertex AI and access to the foundation model.
* Vertex AI initialization : aiplatform.init(..)
  * https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization

In [6]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
import vertexai.preview.generative_models as generative_models

# Initalizate the current vertex AI execution environment.
vertexai.init(project=PROJECT_ID, location=REGION)


system_instructions = [
 "Use Korean when responding",
 "Use table format to categorize the facts"
]

# Access to the generative model.
model = GenerativeModel(MODEL_NAME,
                        system_instruction=system_instructions)

### Function to get the response

In [7]:
def generate(query:str):
    """
    Generate a response from the model.

    query : str
      query to be sent to the model

    Returns: str
      The generated response.

    """

    # Set model parameter : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-multimodal-prompts#set_model_parameters
    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
    }

    # Configure satey setting : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes
    # Refer to the link to remove : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes#how_to_remove_automated_response_blocking_for_select_safety_attributes
    safety_settings = {
        generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    }

    responses = model.generate_content(
        [query],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=False,
    )

    return responses.text

## Run example

In [8]:

from time import perf_counter

t1_start = perf_counter()

prompt = """
You are a Generative AI specialist to help people understand the concept of AI.
Explain what's the pros and cons of using AI in the following instructions.
1. Define what's the Generative AI.
2. What are differences between typical machine learning and Generative AI.
3. What are the pros and cons of using Generative AI.
4. Explain use cases of Generative AI in the financial industry.

"""

outcome = generate(prompt)

t1_end  = perf_counter()
print(f"Time : {t1_end - t1_start} seconds\n\n")

display(Markdown(outcome))



Time : 6.680274750999956 seconds




## 생성형 AI 개념 및 장단점 설명

### 1. 생성형 AI 정의

| 용어 | 설명 |
|---|---|
| 생성형 AI |  새로운 데이터를 생성하는 데 사용되는 인공지능의 한 유형입니다. 텍스트, 이미지, 오디오, 비디오와 같은 다양한 형식의 데이터를 생성할 수 있습니다. |

### 2. 일반적인 머신러닝과 생성형 AI의 차이점

| 특징 | 일반적인 머신러닝 | 생성형 AI |
|---|---|---|
| 목표 | 기존 데이터를 기반으로 예측 또는 분류 | 새로운 데이터 생성 |
| 학습 방식 | 지도 학습, 비지도 학습, 강화 학습 |  생성적 적대 신경망(GAN), 변형 오토인코더(VAE) 등 |
| 예시 | 이미지 분류, 스팸 필터링 | 이미지 생성, 텍스트 작성 |

### 3. 생성형 AI 사용의 장단점

| 장점 | 단점 |
|---|---|
| **창의성 증진:** 새로운 아이디어와 컨텐츠 생성 가능 | **가짜 정보 생성 위험:**  허위 정보 또는 가짜 뉴스 생성 가능 |
| **생산성 향상:** 자동화된 컨텐츠 생성을 통한 효율성 증대 | **데이터 편향 문제:** 학습 데이터의 편향이 생성 결과에 반영될 수 있음 |
| **개인화된 경험 제공:** 사용자 맞춤형 컨텐츠 생성 가능 | **윤리적 문제:**  저작권 침해, 개인정보 유출 위험 등 |
| **새로운 시장 창출:**  새로운 제품 및 서비스 개발 가능 | **보안 취약점:**  생성된 컨텐츠의 보안 위협 |

### 4. 금융 분야에서의 생성형 AI 활용 사례

| 활용 분야 | 예시 |
|---|---|
| **금융 상품 개발:**  고객 맞춤형 금융 상품 설계 및 추천 | 고객 데이터 분석을 기반으로 개인화된 투자 포트폴리오 제안 |
| **위험 관리:**  예측 모델 개발 및 위험 요소 분석 | 금융 시장 변동성 예측 및 리스크 관리 전략 수립 |
| **고객 서비스:**  챗봇 및 가상 비서 개발 | 고객 문의 자동 응답 및 개인화된 금융 상담 제공 |
| **사기 방지:**  사기 패턴 감지 및 예방 | 비정상적인 거래 패턴 분석 및 사기 행위 차단 | 
